## 1. Preface :
1. In this Notebook , I Have tried to find Best Hyperparameters for Light Gbm Model using Bayesian Optimization 
2. I have Taken only 2 million Data Points ! I will try to check out what is best size of Data Points 


## 2. Acknowledgements 
1. To Understand Bayesian Optimization I followed This [Blog](https://medium.com/vantageai/bringing-back-the-time-spent-on-hyperparameter-tuning-with-bayesian-optimisation-2e21a3198afb) , it is very detailed and Explain , Everything Needed for Bayesian Optimization

2. For Code on Bayesian Optimization and Hyperparameters Tuning I refer This  [Kernel](https://www.kaggle.com/somang1418/tuning-hyperparameters-under-10-minutes-lgbm) by Somang So Han ! 

3. For Basic Feature Engineering I refered this [Notebook](https://www.kaggle.com/jsylas/riiid-lgbm-starter) by Sylas

#### Please Do Upvote This Notebook If You Find it Useful It will motivate me to produce More Public Work for this and Upcoming Competitions in Future !

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys
from sklearn.metrics import roc_auc_score, roc_curve
import shap
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import dask.dataframe as dd
from sklearn.metrics import roc_auc_score
import riiideducation
env = riiideducation.make_env()
train = pd.read_csv('../input/riiid-test-answer-prediction/train.csv' , usecols = [1,2,3,4,7,8,9] , 
                   dtype ={'timestamp':'int64' , 'user_id':'int32' , 'content_id':'int16' ,'content_type_id':'int8',
                          'answered_correctly':'int8','prior_question_elapsed_time': 'float32','prior_question_had_explanation': 'boolean'} ,nrows = 2000000)

print("Shape of train set: ",train.shape)
train = train[train.content_type_id == False] # only take question data

train.sort_values(['timestamp'] , ascending = True)

train.drop(columns = ['timestamp' , 'content_type_id'] , axis = 1 ,inplace = True) # drop useless columns

results_c = train[['content_id','answered_correctly']].groupby('content_id').agg('mean') # content_id based mean

results_c.columns = ['answered_correctly_content']


results_u = train[['user_id','answered_correctly']].groupby('user_id').agg(['mean' , 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']


In [ ]:
train.head()

In [ ]:
# Questions.csv 

questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv' , usecols = [0,1,3,4] , dtype = {'question_id': 'int16',
                              'part': 'int8','bundle_id': 'int8','tags': 'str'
})
tags = questions_df['tags'].str.split(" " , n = 10 , expand = True )
# Simply Converts Tags to 5 Columns for Each Questions
# Each Entry in Tag Denotes Value of Tag

In [ ]:
tags.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']
questions_df =  pd.concat([questions_df,tags],axis=1)
questions_df.head()

In [ ]:
# Simply Converting Each Tag to Numeric 
# Coerce is use to set invalid tag values to Nan
questions_df['tags1'] = pd.to_numeric(questions_df['tags1'], errors='coerce')
questions_df['tags2'] = pd.to_numeric(questions_df['tags2'], errors='coerce')
questions_df['tags3'] = pd.to_numeric(questions_df['tags3'], errors='coerce')
questions_df['tags4'] = pd.to_numeric(questions_df['tags4'], errors='coerce')
questions_df['tags5'] = pd.to_numeric(questions_df['tags5'], errors='coerce')
questions_df['tags6'] = pd.to_numeric(questions_df['tags6'], errors='coerce')

In [ ]:
# Train and Validation
X = train.iloc[: , : ]
X['prior_question_had_explanation'].fillna(False , inplace = True)
X  = pd.merge(X , results_c , on = ['content_id'] , how = 'left')
X  = pd.merge(X , results_u , on = ['user_id'] , how = 'left')
X = pd.merge(X , questions_df , left_on = 'content_id' , right_on ='question_id' , how = 'left')
X=X[X.answered_correctly!= -1 ] # only take non Null action Data Points
X  = X.sort_values(['user_id'])
Y = X[['answered_correctly']]
X = X.drop(["answered_correctly"], axis=1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from  sklearn.model_selection import train_test_split

lb_make = LabelEncoder()
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])


X = X[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3','tags4','tags5','tags6']] 
X.fillna(0.5 , inplace = True)



In [ ]:
del train 

In [ ]:
%%time

def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=3, random_seed=6,n_estimators=10000, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=Y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0),
                                            'num_leaves': (24, 80),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,90),
                                            'min_data_in_leaf': (20, 80),
                                            'min_sum_hessian_in_leaf':(0,100),
                                           'subsample': (0.01, 1.0)}, random_state=200)

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    # return best parameters
    return lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']

opt_params = bayes_parameter_opt_lgb(X, Y, init_round=5, opt_round=10, n_folds=3, random_seed=6,n_estimators=10000)

In [ ]:
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['objective']='binary'
opt_params[1]['metric']='auc'
opt_params[1]['is_unbalance']=True
opt_params[1]['boost_from_average']=False
opt_params=opt_params[1]
opt_params

In [ ]:
%%time 

target= Y['answered_correctly']
features= [c for c in X.columns]


folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=31416)
oof = np.zeros(len(X))
#predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(X.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 15000
    clf = lgb.train(opt_params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 250)
    oof[val_idx] = clf.predict(X.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))


In [ ]:
import pickle
filename = 'lgbm_2m_bayesian_hyperopt_tuned.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# Making Final Predictions on Dtest
iter_test = env.iter_test()
batch = 0 
for (test_df , sample_prediction_df) in iter_test:
    test_df = test_df.sort_values(['user_id','timestamp'], ascending=False)
    test_df['answer_time'] = test_df.groupby(['user_id'])['prior_question_elapsed_time'].shift(1)
    # Features for Test Set
    test_df = pd.merge(test_df , results_u , on = ['user_id'] , how = 'left')
    test_df = pd.merge(test_df , results_c , on = ['content_id'] , how = 'left')
    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')  
    # Imputations and Transformations
    test_df['answered_correctly_user'].fillna(0.5, inplace=True)
    test_df['answered_correctly_content'].fillna(0.5, inplace=True)
    test_df['sum'].fillna(0, inplace=True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    # Dtest Predictions
    test_df['answered_correctly'] =  clf.predict(test_df[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc',
                                                           'tags1','tags2','tags3','tags4','tags5','tags6']])
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    print("Batch Done : ", batch)
    batch += 1

### Please Do Checkout Some of My Other Works in this Competition !
1. EDA and Problem Understanding Kernel [Here](https://www.kaggle.com/sayedathar11/riiid-eda-let-s-explore) 
2. Simple User Id Mean Correctness  Baseline [Here](https://www.kaggle.com/sayedathar11/riiid-mean-user-accuracy-baseline) 